In [1]:
import sys
import io
import os

import numpy as np
from random import randint
from random import random

from collections import *

import re

from os import listdir
from os.path import isfile, join

In [2]:
def clean_text(text):
    '''Clean text by removing unnecessary characters and altering the format of words.'''

    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"'s", " is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[”“’‘'-()\"#/@;:<>{}`+=~|!?]", "", text)
    
    return text

In [3]:
### Read/process inputs
input_path = './raw_data/'
all_data_sents = []

files = [join(input_path, f) for f in listdir(input_path) if isfile(join(input_path, f))]
for filename in files:
    if filename.endswith('.txt'):
        with open(filename, encoding='utf-8', errors='ignore') as f:
            text = f.read().lower()
            text = text.replace('\n\n', '\n')\
                .replace('"', '')\
                .replace('-', ' - ')\
                .replace("\"", '')\
                .replace("...", "")\
                .replace("…", "")\
                .replace("—", "")\
                .replace(". ", " . ")\
                .replace(", ", " , ")
            text = ' '.join(text.split("\n"))
            all_data_sents.append(text)
    
print(len(all_data_sents))

all_data_string = ' '.join(all_data_sents)
print(len(all_data_string))

all_data_string = clean_text(all_data_string)
print(len(all_data_string))

5
9609400
9534701


In [4]:
# Prepare test set
TEST_SET_SIZE = 20

sents = [sent for sent in all_data_string.split('.') if sent.strip() != '' or sent == '\n']
train_set, test_set = sents, []

for x in range(TEST_SET_SIZE):
    index = randint(0, len(sents))
    test_set.append(train_set.pop(index).strip())
    
train_set = '.'.join(train_set)

In [5]:
def xrange(x):
    return iter(range(x))

def train_char_lm(data, order=4):
    lm = defaultdict(Counter)
    pad = "~" * order
    data = pad + data
    for i in xrange(len(data)-order):
        history, char = data[i:i+order], data[i+order]
        lm[history][char]+=1
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

In [6]:
def generate_letter(lm, history, order):
        history = history[-order:]
        dist = lm[history]
        x = random()
        for c,v in dist:
            x = x - v
            if x <= 0: return c

In [7]:
def generate_text(lm, order, history=None, nletters=1000):
    if history is None:
        history = "~" * order
    else:
        history = history[0:order]
        if len(history) < order:
            history = ("~" * (order - len(history))) + history
    out = []
    for i in xrange(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
    return "".join(out)

In [8]:
ORDER = 20

lm = train_char_lm(all_data_string, ORDER)

In [ ]:
test_index = randint(0, TEST_SET_SIZE)
test_input = test_set[test_index]
print("TEST INPUT:\n{}\n".format(test_input))
print("GENERATED TEXT:\n")
print(generate_text(lm, ORDER, test_input))

In [ ]:
print(generate_text(lm, ORDER, 'tyrion drank it in his window seat,'))

In [9]:
print(generate_text(lm, ORDER, 'this is a test sentence this is just a test'))

KeyError: 'this is a test sente'